<a href="https://colab.research.google.com/github/pv21287/CE888/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the libraries needed

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import joblib

# Load the dataset

In [ ]:
df=pd.read_csv('train.csv')
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [ ]:
df['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [ ]:
df['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [ ]:
len(df)

891

In [ ]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [ ]:
df['Sex']=pd.factorize(df['Sex'])[0]
df['Pclass']=pd.factorize(df['Pclass'])[0]
df['Embarked']=pd.factorize(df['Embarked'])[0]

# Create the training and test sets

In [ ]:
X=df.drop(['Survived'], axis=1)
y=df['Survived']
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis=1), df['Survived'], 
                                                    test_size=0.3, random_state=42, stratify=y)

# Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(X)

In [ ]:
from sklearn.impute import SimpleImputer


imp=SimpleImputer() 
imp_X_train = pd.DataFrame(imp.fit_transform(X_train))
imp_X_test = pd.DataFrame(imp.transform(X_test))


imp_X_train.columns = X_train.columns
imp_X_test.columns = X_test.columns

In [ ]:
X = pd.concat([imp_X_train, imp_X_test])
y = pd.concat([y_train, y_test])


In [ ]:
len(X.Pclass)

891

# Train a Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42, min_samples_split=10)
classifier.fit(imp_X_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=10,
                       n_estimators=200, random_state=42)

In [ ]:
accuracy_score(y, classifier.predict(X))

0.8810325476992144

In [ ]:
accuracy_score(y_train, classifier.predict(imp_X_train))

0.9085072231139647

In [ ]:
accuracy_score(y_test, classifier.predict(imp_X_test))

0.8171641791044776

# Obtain the Probabilities for classes

In [ ]:
prediction_probs=classifier.predict_proba(X)

In [ ]:
len(prediction_probs)

891

In [ ]:
len(prediction_probs[0])

2

In [ ]:
df['probs_1']=[prediction_probs[i][1] for i in range(len(df.Survived))]

In [ ]:
df['probs_1']=[df['probs_1'][i]>=0.5 for i in range(len(df.Survived))]*1

In [ ]:
df.probs_1

0      False
1      False
2       True
3      False
4      False
       ...  
886     True
887     True
888    False
889    False
890     True
Name: probs_1, Length: 891, dtype: bool

In [ ]:
df.probs_1.value_counts()

False    595
True     296
Name: probs_1, dtype: int64

# Decision Tree Classifier trained on the obtained results from RF model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# y_tr=df.probs_1[y_train.index]
# y_ts=df.probs_1[y_test.index]


dt=DecisionTreeClassifier(max_depth=None, min_samples_leaf=1)

dt.fit(X, df.probs_1)


DecisionTreeClassifier()

In [ ]:
accuracy_score(y, dt.predict(X))

0.8810325476992144

In [ ]:
accuracy_score(y_train, dt.predict(imp_X_train))

0.9085072231139647

In [ ]:
accuracy_score(y_test, dt.predict(imp_X_test))

0.8171641791044776

In [ ]:
accuracy_score(df.probs_1, dt.predict(X))

1.0

In [ ]:
accuracy_score(y, dt.predict(X))

0.8810325476992144

In [ ]:
dt.predict(imp_X_test)

array([False, False, False, False, False, False, False,  True, False,
        True,  True,  True, False, False,  True,  True, False, False,
        True,  True, False, False,  True, False, False, False,  True,
       False, False, False,  True,  True, False,  True,  True, False,
       False, False, False, False, False,  True, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False, False, False,  True, False, False,  True, False,
       False,  True, False, False, False, False, False, False, False,
        True,  True, False, False, False, False,  True, False, False,
        True, False, False,  True, False,  True,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False, False,

In [ ]:
accuracy_score(y_train, dt.predict(imp_X_train))

0.9085072231139647

# Decision Tree Classifier trained on the original data

In [ ]:
dt1=DecisionTreeClassifier(max_depth=None, min_samples_leaf=1)

dt1.fit(imp_X_train, y_train)

DecisionTreeClassifier()

In [ ]:
accuracy_score(y_train, dt1.predict(imp_X_train))

0.9807383627608347

In [ ]:
accuracy_score(y_test, dt1.predict(imp_X_test))

0.7649253731343284

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,probs_1
0,0,0,0,22.0,1,0,7.2500,0,False
1,1,1,1,38.0,1,0,71.2833,1,False
2,1,0,1,26.0,0,0,7.9250,0,True
3,1,1,1,35.0,1,0,53.1000,0,False
4,0,0,0,35.0,0,0,8.0500,0,False


In [ ]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,0.0,19.000000,1.0,0.0,53.1000,0.0
1,0.0,0.0,30.045476,0.0,0.0,8.0500,0.0
2,0.0,1.0,30.045476,0.0,0.0,7.8792,2.0
3,1.0,0.0,30.045476,0.0,0.0,0.0000,0.0
4,0.0,0.0,28.000000,1.0,0.0,15.8500,0.0
...,...,...,...,...,...,...,...
263,0.0,1.0,4.000000,0.0,2.0,22.0250,0.0
264,1.0,0.0,27.000000,0.0,0.0,30.5000,0.0
265,0.0,0.0,21.000000,0.0,0.0,16.1000,0.0
266,0.0,0.0,33.000000,0.0,0.0,8.6542,0.0
